In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import random_split
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import os

MODEL_NUMBER = 4

    
# seed so random stuff isnt random
SEED = 100

# transforms
transform = transforms.Compose([

    transforms.RandomResizedCrop(64, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
])

dataset = torchvision.datasets.EuroSAT(root='./data', download=True, transform=transform)

# split into train, test
# 80% train, 20% test 
train_ratio = 0.8
test_ratio = 0.2


total_size = len(dataset)
train_size = int(total_size * train_ratio)
test_size = total_size - train_size


train_dataset, test_dataset = random_split(
    dataset, [train_size, test_size],
    generator=torch.Generator().manual_seed(SEED)
)

print(f"train: {len(train_dataset)}")
print(f"test size: {len(test_dataset)}")

train: 21600
test size: 5400


In [2]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        self.relu = torch.nn.ReLU()

        self.c1 = torch.nn.Conv2d(3, 8, 3, padding=1)
        self.c2 = torch.nn.Conv2d(8, 32, 3, padding=1)
        self.c3 = torch.nn.Conv2d(32, 64, 3, padding=1)
        self.maxpool1 = torch.nn.MaxPool2d(2)
        self.c4 = torch.nn.Conv2d(64, 128, 3, padding=1)
        self.maxpool2 = torch.nn.MaxPool2d(2)
        self.c5 = torch.nn.Conv2d(128, 256, 3, padding=1)
        self.flatten = torch.nn.Flatten()
        
        self.fc1 = torch.nn.Linear(256 * 16 * 16, 1024)
        self.dropout1 = torch.nn.Dropout(0.5)
        self.fc2 = torch.nn.Linear(1024, 256)
        self.dropout2 = torch.nn.Dropout(0.5)
        self.fc3 = torch.nn.Linear(256, 10)

    def forward(self, x):
        x = self.c1(x)
        x = self.relu(x)
        x = self.c2(x)
        x = self.relu(x)
        x = self.maxpool1(x)
        x = self.c3(x)
        x = self.relu(x)
        x = self.c4(x)
        x = self.relu(x)
        x = self.maxpool2(x)
        x = self.c5(x)
        x = self.relu(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

model = Model()
print(f"Model: {model}")
print(f"Total parameters: {sum(p.numel() for p in model.parameters())}")



Model: Model(
  (relu): ReLU()
  (c1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (c2): Conv2d(8, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (c3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (c4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (c5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=65536, out_features=1024, bias=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=1024, out_features=256, bias=True)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
)
Total parameters: 67764938


In [3]:

# dataloaders
train_loader = torch.utils.data.DataLoader(
    train_dataset, 
    batch_size=128, 
    shuffle=True,
    pin_memory=True, # for use on gpu
    num_workers=4 # arbitrarily chosen. some sources recommend 4 per gpu
)


test_loader = torch.utils.data.DataLoader(
    test_dataset, 
    batch_size=128, 
    shuffle=False,
    pin_memory=True,
    num_workers=4
)

In [4]:

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

device = "cuda" if torch.cuda.is_available() else "cpu"

if device == "cuda":
    model.to(device, non_blocking=True) # according to pytorch docs we can do this if memory is pinned
else:  
    model.to(device)
    


In [5]:
def train_one_epoch():
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    avg_loss = total_loss / len(train_loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy

def test_one_epoch():
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    avg_loss = total_loss / len(test_loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy

In [6]:
# tensorboard setup
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/train_{}_{}'.format(MODEL_NUMBER, timestamp))

checkpoint_dir = f'checkpoints/train_{MODEL_NUMBER}_{timestamp}'
os.makedirs(checkpoint_dir, exist_ok=True)

EPOCHS = 200

for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    


    train_loss, train_acc = train_one_epoch()
    
    test_loss, test_acc = test_one_epoch()
    
    print(f'train loss: {train_loss:.4f} | test acc: {train_acc:.2f}%')
    print(f'test loss:  {test_loss:.4f} | test acc:  {test_acc:.2f}%')
    
    # log data
    writer.add_scalar('loss/1_train', train_loss, epoch + 1)
    writer.add_scalar('loss/2_test', test_loss, epoch + 1)
    writer.add_scalar('accuracy/1_train', train_acc, epoch + 1)
    writer.add_scalar('accuracy/2_test', test_acc, epoch + 1)
    writer.flush()
    
    # checkpoint every 20 epochs
    if (epoch + 1) % 20 == 0:
        checkpoint_path = os.path.join(checkpoint_dir, f'model_epoch_{epoch + 1}.pt')
        torch.save(model.state_dict(), checkpoint_path)
        print(f'checkpoint saved: {checkpoint_path}')
    
    print()

# save final model
final_checkpoint_path = os.path.join(checkpoint_dir, f'model_final.pt')
torch.save(model.state_dict(), final_checkpoint_path)
print(f'final checkpoint saved: {final_checkpoint_path}')

writer.close()
print('Complete')

Epoch 1/200
train loss: 1.8162 | test acc: 29.27%
test loss:  1.5721 | test acc:  38.35%

Epoch 2/200
train loss: 1.2666 | test acc: 52.45%
test loss:  1.0119 | test acc:  62.24%

Epoch 3/200
train loss: 1.0532 | test acc: 61.44%
test loss:  1.0834 | test acc:  59.89%

Epoch 4/200
train loss: 0.9383 | test acc: 65.88%
test loss:  0.8940 | test acc:  66.19%

Epoch 5/200
train loss: 0.8689 | test acc: 68.61%
test loss:  0.7655 | test acc:  71.04%

Epoch 6/200
train loss: 0.7934 | test acc: 71.40%
test loss:  0.7172 | test acc:  73.72%

Epoch 7/200
train loss: 0.7363 | test acc: 73.56%
test loss:  0.6779 | test acc:  75.35%

Epoch 8/200
train loss: 0.7140 | test acc: 74.73%
test loss:  0.6536 | test acc:  76.30%

Epoch 9/200
train loss: 0.6706 | test acc: 76.46%
test loss:  0.6471 | test acc:  76.70%

Epoch 10/200
train loss: 0.6641 | test acc: 77.08%
test loss:  0.5856 | test acc:  78.56%

Epoch 11/200
train loss: 0.6031 | test acc: 78.83%
test loss:  0.5490 | test acc:  80.35%

Epoch 12